In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd;

In [ ]:
ttData = pd.read_csv('/kaggle/input/top-100-world-ranking-tiktok-accounts-in-2025/Top 100 tiktokers in 2025.csv')


In [ ]:
ttData.head(10)

# RELATIONSHIP BETWEEN FOLLOWERS AND UPLOADS

In [ ]:
corrFollowers_Uploads = ttData['Followers'].corr(ttData['Uploads'])
print(corrFollowers_Uploads)

if corrFollowers_Uploads > 0.7:
    print("There is a strong positive correlation between followers and uploads")
elif corrFollowers_Uploads < -0.7:
    print("There is a strong negative correlation between followers and uploads")
elif -0.7 <= corrFollowers_Uploads <= 0.7:
    print("There is a weak or no significant relationship between followers and uploads")



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.scatterplot(x='Uploads',y='Followers',data=ttData)
plt.title("Correlation between followers and uploads")
plt.xlabel("Uploads")
plt.ylabel("Followers")
plt.show()

##### OBSERVATIONS
##### Most tiktokers have fewer than 5000 uploads and less that a million followers
##### Some users have very high followers but low number of uploads
##### So relationship is weak

# RELATIONSHIP BETWEEN LIKES AND UPLOADS

In [ ]:
likesUploadscorr=ttData['Uploads'].corr(ttData['Likes'])
print(likesUploadscorr)

if likesUploadscorr > 0.7:
    print("There is a strong positive correlation between followers and uploads")
elif likesUploadscorr < -0.7:
    print("There is a strong negative correlation between followers and uploads")
elif -0.7 <= likesUploadscorr <= 0.7:
    print("There is a weak or no significant relationship between followers and uploads")

In [ ]:
sns.scatterplot(x='Uploads',y='Likes',data=ttData)
plt.title("Correlation between likes and uploads")
plt.xlabel("Uploads")
plt.ylabel("Likes")
plt.show()

#### OBSERVATIONS 
Most tiktokers have less than 5000 uploads and less likes
Some users have high number of uploads and more likes

# RELATIONSHIP BETWEEN FOLLOWERS AND LIKES

In [ ]:
followerLikescorr=ttData['Followers'].corr(ttData['Likes'])
if followerLikescorr > 0.7:
    print("There is a strong positive correlation between followers and uploads")
elif followerLikescorr < -0.7:
    print("There is a strong negative correlation between followers and uploads")
elif -0.7 <= followerLikescorr <= 0.7:
    print("There is a weak or no significant relationship between followers and uploads")

In [ ]:
sns.scatterplot(x='Followers',y='Likes',data=ttData)
plt.title("Correlation between likes and followers")
plt.xlabel("Followers")
plt.ylabel("Likes")
plt.show()

##### OBSERVATIONS
-Most people have less than a million followers and even less likes
(Are people buying followers?)

# K-MEANS CLUSTERING

## Pre-processing


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
data=ttData[['Followers','Uploads','Likes']]
scaler=StandardScaler()
dataScaled=scaler.fit_transform(data)

In [ ]:
from sklearn.cluster import KMeans
inertia=[]
for k in range(1,11):
    kmeans=KMeans(n_clusters=k)
    kmeans.fit(dataScaled)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
kmeans=KMeans(n_clusters=5)
data['Cluster']=kmeans.fit_predict(dataScaled)

plt.scatter(data['Followers'], data['Uploads'], c=data['Cluster'], cmap='viridis')
plt.xlabel('Followers')
plt.ylabel('Uploads')
plt.title('K-means Clusters of Followers vs Uploads')
plt.show()

In [ ]:

print("Cluster centers:", kmeans.cluster_centers_)

print(data.groupby('Cluster').mean())


### ANALYZING CLUSTERS

In [ ]:
print(data['Cluster'].value_counts())

### FOLLOWERS VS LIKES FOR CLUSTER 0

In [ ]:
cluster0Data=data[data['Cluster']==0]
plt.scatter(cluster0Data["Followers"],cluster0Data["Likes"],alpha=0.7)
plt.xlabel("Followers")
plt.ylabel("Likes")
plt.title("Examining Cluster 0")
plt.show()

### RECLUSTERING CLUSTER 0


In [ ]:
cluster0Scaled=scaler.fit_transform(cluster0Data)

In [ ]:
inertia=[]
for k in range (1,9):
    kmeans=KMeans(n_clusters=k,random_state=42)
    kmeans.fit(cluster0Scaled)
    inertia.append(kmeans.inertia_)
plt.plot(range(1, 9), inertia, marker='o')
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.title("Elbow Method for Sub-Clustering Cluster 0")
plt.show()

In [ ]:
optimal_k = 4 
kmeans_0 = KMeans(n_clusters=optimal_k, random_state=42)
cluster0Data["SubCluster"] = kmeans_0.fit_predict(cluster0Scaled)


sns.scatterplot(x=cluster0Data["Followers"], 
                y=cluster0Data["Likes"], 
                hue=cluster0Data["SubCluster"], 
                palette="viridis")
plt.xlabel("Followers")
plt.ylabel("Likes")
plt.title("Sub-Clusters within Cluster 0")
plt.show()

In [ ]:
print(cluster0Data.groupby("SubCluster").mean())

### SUBCLUSTERING THEM BACK TO MAIN CLUSTERS

In [ ]:

data.loc[data["Cluster"] == 0, "Cluster"] = cluster0Data["SubCluster"].values


In [ ]:
plt.scatter(data["Followers"], data["Uploads"], c=data["Cluster"], cmap="viridis")
plt.xlabel("Followers")
plt.ylabel("Uploads")
plt.title("Final Clusters After Refinement")
plt.show()


In [ ]:
plt.scatter(data["Followers"], data["Likes"], c=data["Cluster"], cmap="viridis")
plt.xlabel("Followers")
plt.ylabel("Likes")
plt.title("Final Clusters")
plt.show()

In [ ]:
plt.scatter(data["Uploads"], data["Likes"], c=data["Cluster"], cmap="viridis")
plt.xlabel("Uploads")
plt.ylabel("Likes")
plt.title("Final Clusters After Refinement")
plt.show()

In [ ]:
print(data['Cluster'].value_counts())